<a href="https://colab.research.google.com/github/DiscoveredGoal/soccer-line-detection/blob/main/soccer-line-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/DiscoveredGoal/soccer-line-detection
!pip install import_ipynb
!pip install easyocr
!pip install opencv-python-headless==4.1.2.30
!pip install options
!pip install scipy==1.1.0
!pip install dominate

Cloning into 'soccer-line-detection'...
remote: Enumerating objects: 929, done.
remote: Counting objects: 100% (929/929), done.
remote: Compressing objects: 100% (911/911), done.
remote: Total 929 (delta 14), reused 922 (delta 12), pack-reused 0
Receiving objects: 100% (929/929), 19.43 MiB | 26.63 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [3]:
import torch
print("the version of the torch used is" , torch.__version__)
print("status of the cuda gpu", torch.cuda.is_available())

the version of the torch used is 1.10.0+cu111
status of the cuda gpu True


In [2]:
# !git clone https://github.com/pytorch/vision
# !git checkout v0.11.1
# !python setup.py install

Cloning into 'vision'...
remote: Enumerating objects: 110617, done.
remote: Counting objects: 100% (3302/3302), done.
remote: Compressing objects: 100% (765/765), done.
remote: Total 110617 (delta 2980), reused 2730 (delta 2520), pack-reused 107315
Receiving objects: 100% (110617/110617), 214.40 MiB | 28.66 MiB/s, done.
Resolving deltas: 100% (95640/95640), done.


In [10]:
!ls

category_names.txt  main.ipynb			 test.py
checkpoints	    models			 test.sh
data		    options			 test_two_pix2pix.py
datasets	    pretrained_models		 train.py
generate_line.py    README.md			 train.sh
generate.sh	    results			 train_two_pix2pix.py
imgs		    scripts			 util
LICENSE		    soccer-line-detection.ipynb


In [1]:
!python train_two_pix2pix.py --dataroot ./datasets/soccer_seg_detection --name soccer_seg_detection_pix2pix --model two_pix2pix --which_model_netG unet_256 --which_direction AtoB --lambda_A 100 --dataset_mode two_aligned --no_lsgan --norm batch --pool_size 0 --output_nc 1 --phase1 train_phase_1 --phase2 train_phase_2 --save_epoch_freq 2

python3: can't open file 'train_two_pix2pix.py': [Errno 2] No such file or directory


In [2]:
cd soccer-line-detection/

/content/soccer-line-detection


In [8]:
import math
import cv2
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import matplotlib.patches as patches
import os
import torch
import torchvision.transforms as transforms
import skimage

from options.generate_options import GenerateOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
from util.visualizer import Visualizer
from util.util import save_image
from PIL import Image



In [4]:

precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
ssd_model = ssd_model.to('cuda')
ssd_model.eval()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


SSD300(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplac

In [5]:
def SSD(tensor, ssd_model):
    with torch.no_grad():
        detections = ssd_model(tensor)
    results = utils.decode_results(detections)
    best_results = [utils.pick_best(results, 0.40) for results in results]
    classes_to_labels = utils.get_coco_object_dictionary()
    person_counter = 0
    for image_idx in range(len(best_results)):
#         fig, ax = plt.subplots(1, figsize=(7,7))
        # Show original, denormalized image...
        image = tensor[image_idx] / 2 + 0.5
#         ax.imshow(image)
        # ...with detections
        bboxes, classes, confidences = best_results[image_idx]
        for idx in range(len(bboxes)):
#             left, bot, right, top = bboxes[idx]
#             x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
#             rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
#             ax.add_patch(rect)
#             ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
            if classes[idx] == 1:
                person_counter += 1
#         plt.show()
    return person_counter

In [6]:
def load_PIL_image(PIL_image):
    """Code from Loading_Pretrained_Models.ipynb - a Caffe2 tutorial"""
    img = skimage.img_as_float(PIL_image)
    if len(img.shape) == 2:
        img = np.array([img, img, img]).swapaxes(0,2)
    return img

def prepare_input_from_PIL(PIL_img):
    img = load_PIL_image(PIL_img)
    img = utils.rescale(img, 300, 300)
    img = utils.crop_center(img, 300, 300)
    img = utils.normalize(img)

    return img

In [7]:
def is_longview(img, ssd_model) -> bool:
    img = cv2.resize(img, (455, 256))
    cv2.imshow("input", img)
    height, width, _ = img.shape
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    thres = 0.75
    h_lower = 30
    h_upper = 90
    
    field_mask = cv2.inRange(hsv, (h_lower, 0, 50), (h_upper, 255,255))
    persons = 5
    is_field = (np.sum(field_mask)/(width*height)/255 > thres)
#     if is_field:
#         inputs = [prepare_input_from_PIL(img)]
#         tensor = utils.prepare_tensor(inputs, precision == 'fp16').to('cuda')
#         persons = SSD(tensor, ssd_model)
    
    return is_field and persons > 3

In [2]:
opt = GenerateOptions()

opt.isTrain = False
opt.dataroot = "./datasets/generate"
opt.batchSize = 1
opt.loadSize = 256
opt.fineSize = 256
opt.input_nc = 3
opt.output_nc = 1
opt.ngf = 64
opt.ndf = 64
opt.which_model_netD = "basic"
opt.which_model_netG = "unet_256"
opt.n_layers_D = 3
opt.gpu_ids = [0]
opt.name = "soccer_seg_detection_pix2pix"
opt.dataset_mode = "single"
opt.model = "generate"
opt.which_direction = "AtoB"
opt.nThreads = 1
opt.checkpoints_dir = "./checkpoints"
opt.norm = "batch"
opt.serial_batches = True
opt.display_winsize = 256
opt.display_id = 1
opt.display_port = 8097
opt.no_dropout = True
opt.max_dataset_size = float("inf")
opt.resize_or_crop = 'resize_and_crop'
opt.no_flip = True
opt.init_type = 'normal'

opt.continue_train = False
opt.ntest = float("inf")
opt.results_dir = './'
opt.aspect_ratio = 1.0
opt.phase = "generate"
opt.which_epoch = "latest"
opt.how_many = 1
opt.output_dir = './'



# data_loader = CreateDataLoader(opt)
# dataset = data_loader.load_data()
model = create_model(opt)
# visualizer = Visualizer(opt)

counter = 1

NameError: ignored